In [1]:
import sqlite3
import pandas as pd
from pandas.core.frame import DataFrame
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None

from matplotlib import pyplot as plt
from time import sleep

import matplotlib


# import gnuplotlib as gp
import numpy as np

import sys

In [2]:
pd.set_option('display.max_colwidth', None)

In [3]:
import requests

In [4]:
def get_df(f_path: str = "/home/kefei/.volttron/agents/"
                       "ba3caba2-2e44-49a3-a771-eaccf6b7bc3e/sqlhistorianagent-4.0.0/data/platform.historian.sqlite"
           ) -> pd.DataFrame:
    # f_path = "/home/kefei/.volttron/agents/85736715-3354-4b70-94d7-eb2ca6e88e48/sqlhistorianagent-4.0.0/data/platform.historian.sqlite"
    con = sqlite3.connect(f_path)

    # df = pd.read_sql_query("SELECT name FROM sqlite_schema WHERE type ='table' AND name NOT LIKE 'sqlite_%'", con)
    # df.head()

    df_data = pd.read_sql_query(sql="select * from data", con=con)
    # df_data

    df_topics = pd.read_sql_query(sql="select * from topics", con=con)
    # df_topics
    con.commit()  # to make sure refresh

    df_join = pd.merge(df_data, df_topics, on='topic_id')
    # df_join.head()

    return df_join

In [5]:
df = get_df()
df.tail()

,ts,topic_id,value_string,topic_name,metadata
74251,2022-05-19T23:45:28.927319+00:00,92,false,campus-cloud/building-cloud/RestAPI/random_bool,"{""units"": ""1"", ""type"": ""integer"", ""tz"": ""UTC""}"
74252,2022-05-19T23:45:32.966951+00:00,92,true,campus-cloud/building-cloud/RestAPI/random_bool,"{""units"": ""1"", ""type"": ""integer"", ""tz"": ""UTC""}"
74253,2022-05-19T23:45:42.753190+00:00,92,false,campus-cloud/building-cloud/RestAPI/random_bool,"{""units"": ""1"", ""type"": ""integer"", ""tz"": ""UTC""}"
74254,2022-05-19T23:45:43.884460+00:00,92,false,campus-cloud/building-cloud/RestAPI/random_bool,"{""units"": ""1"", ""type"": ""integer"", ""tz"": ""UTC""}"
74255,2022-05-19T23:45:47.841723+00:00,92,true,campus-cloud/building-cloud/RestAPI/random_bool,"{""units"": ""1"", ""type"": ""integer"", ""tz"": ""UTC""}"


In [6]:
# inspect here
df = get_df()
df.sort_values('ts', ascending=False).head(8)


,ts,topic_id,value_string,topic_name,metadata
74255,2022-05-19T23:45:47.841723+00:00,92,true,campus-cloud/building-cloud/RestAPI/random_bool,"{""units"": ""1"", ""type"": ""integer"", ""tz"": ""UTC""}"
74192,2022-05-19T23:45:47.841723+00:00,91,29134.7887,campus-cloud/building-cloud/RestAPI/bitcoin_eur,"{""units"": ""euro"", ""type"": ""integer"", ""tz"": ""UTC""}"
74129,2022-05-19T23:45:47.841723+00:00,90,24754.1429,campus-cloud/building-cloud/RestAPI/bitcoin_gbp,"{""units"": ""pound"", ""type"": ""integer"", ""tz"": ""UTC""}"
74066,2022-05-19T23:45:47.841723+00:00,89,30290.5236,campus-cloud/building-cloud/RestAPI/bitcoin_usd,"{""units"": ""dollar"", ""type"": ""integer"", ""tz"": ""UTC""}"
74191,2022-05-19T23:45:43.884460+00:00,91,29134.7887,campus-cloud/building-cloud/RestAPI/bitcoin_eur,"{""units"": ""euro"", ""type"": ""integer"", ""tz"": ""UTC""}"
74254,2022-05-19T23:45:43.884460+00:00,92,false,campus-cloud/building-cloud/RestAPI/random_bool,"{""units"": ""1"", ""type"": ""integer"", ""tz"": ""UTC""}"
74128,2022-05-19T23:45:43.884460+00:00,90,24739.8092,campus-cloud/building-cloud/RestAPI/bitcoin_gbp,"{""units"": ""pound"", ""type"": ""integer"", ""tz"": ""UTC""}"
74065,2022-05-19T23:45:43.884460+00:00,89,30272.9841,campus-cloud/building-cloud/RestAPI/bitcoin_usd,"{""units"": ""dollar"", ""type"": ""integer"", ""tz"": ""UTC""}"


In [7]:
from IPython.display import display, clear_output

In [12]:
# inspect here

topic_prfix = "campus/building/restapi/"
topic_prfix = "campus/building/user-fake/"
topic_prfix = "campus-cloud/building-cloud/"
# campus-cloud/building-cloud/RestAPI/random_bool 	
# topic_prfix = "/"

df = get_df()
df_filetered = df.sort_values('ts', ascending=False)
df_display = df_filetered[df_filetered.topic_name.str.contains(topic_prfix)].head(8)
df_display

old_index = df_display.index
old_index
# df_display.loc[old_index,:]
df_display.loc[old_index,:]

count = 0
while count < 500:
    df = get_df()
    df_filetered = df.sort_values('ts', ascending=False)
    df_display = df_filetered[df_filetered.topic_name.str.contains(topic_prfix)].head(8)
    df_display
    new_index = df_display.index.copy()
    index_diff = new_index.difference(old_index)
    old_index = new_index
    if len(index_diff) > 0:
#         print(df_filetered.loc[index_diff].head(8))
#         print(df_display)
        display(df_display.loc[index_diff, :])
    else:
        print("no new data published. Count at: ", count)
    count += 1
#     print("count", count)
    sleep(5)
    clear_output(wait=True)

,ts,topic_id,value_string,topic_name,metadata
74135,2022-05-20T00:07:48.028528+00:00,89,30309.539,campus-cloud/building-cloud/RestAPI/bitcoin_usd,"{""units"": ""dollar"", ""type"": ""integer"", ""tz"": ""UTC""}"
74266,2022-05-20T00:07:42.906963+00:00,90,24769.6827,campus-cloud/building-cloud/RestAPI/bitcoin_gbp,"{""units"": ""pound"", ""type"": ""integer"", ""tz"": ""UTC""}"
74267,2022-05-20T00:07:48.028528+00:00,90,24769.6827,campus-cloud/building-cloud/RestAPI/bitcoin_gbp,"{""units"": ""pound"", ""type"": ""integer"", ""tz"": ""UTC""}"
74398,2022-05-20T00:07:42.906963+00:00,91,29153.0785,campus-cloud/building-cloud/RestAPI/bitcoin_eur,"{""units"": ""euro"", ""type"": ""integer"", ""tz"": ""UTC""}"
74399,2022-05-20T00:07:48.028528+00:00,91,29153.0785,campus-cloud/building-cloud/RestAPI/bitcoin_eur,"{""units"": ""euro"", ""type"": ""integer"", ""tz"": ""UTC""}"
74530,2022-05-20T00:07:42.906963+00:00,92,true,campus-cloud/building-cloud/RestAPI/random_bool,"{""units"": ""1"", ""type"": ""integer"", ""tz"": ""UTC""}"
74531,2022-05-20T00:07:48.028528+00:00,92,true,campus-cloud/building-cloud/RestAPI/random_bool,"{""units"": ""1"", ""type"": ""integer"", ""tz"": ""UTC""}"


KeyboardInterrupt: 